In [ ]:
%pip install transformers sentence-transformers numpy scikit-learn


# What Are Embeddings?
Definition: Embeddings are vector representations of text that capture semantic meaning.

Why Use Them?:

To find similarity between texts.

To enable tasks like search, clustering, and classification.

In [ ]:
from sentence_transformers import SentenceTransformer

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example sentences
sentences = [
    "Artificial Intelligence is transforming industries.",
    "Machine learning is a subset of AI."
]

# Generate embeddings
embeddings = model.encode(sentences)

print("Embeddings:")
print(embeddings)
print("Shape of each embedding:", embeddings[0].shape)
